In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyspark.sql.functions as F

# Read and Clean Data

In [2]:
review_df = pd.read_csv("data/reviews_full.csv")
listing_df = pd.read_csv("data/listings_full.csv")

/Users/nathanwong/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (43,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
cleanrev_df = review_df[['listing_id', 'comments']]
cleanrev_df.rename(columns={'listing_id': 'id'}, inplace=True)
cleanrev_df.head()

/Users/nathanwong/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,id,comments
0,2265,I stayed here during SXSW and had a really ple...
1,2265,"Great place, close enough to everything downto..."
2,2265,We had a great time in Austin staying at Paddy...
3,2265,We had a great stay at Zen East for South By S...
4,2265,I arrived late in the evening so did not meet ...


In [4]:
cleanlist_df = listing_df[['id', 'name', 'zipcode', 'review_scores_value']]
cleanlist_df.head()

,id,name,zipcode,review_scores_value
0,2265,Zen-East in the Heart of Austin (monthly rental),78702,9.0
1,5245,"Eco friendly, Colorful, Clean, Cozy monthly share",78702,9.0
2,5456,"Walk to 6th, Rainey St and Convention Ctr",78702,10.0
3,5769,NW Austin Room,78729,10.0
4,6413,Gem of a Studio near Downtown,78704,10.0


In [5]:
full_df = pd.merge(left=cleanrev_df,right=cleanlist_df, on='id')
full_df.head()

,id,comments,name,zipcode,review_scores_value
0,2265,I stayed here during SXSW and had a really ple...,Zen-East in the Heart of Austin (monthly rental),78702,9.0
1,2265,"Great place, close enough to everything downto...",Zen-East in the Heart of Austin (monthly rental),78702,9.0
2,2265,We had a great time in Austin staying at Paddy...,Zen-East in the Heart of Austin (monthly rental),78702,9.0
3,2265,We had a great stay at Zen East for South By S...,Zen-East in the Heart of Austin (monthly rental),78702,9.0
4,2265,I arrived late in the evening so did not meet ...,Zen-East in the Heart of Austin (monthly rental),78702,9.0


In [6]:
train_df = full_df[['comments', 'review_scores_value']]
train_df['review_scores_value'] = train_df['review_scores_value']*0.5
train_df.head()

/Users/nathanwong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,comments,review_scores_value
0,I stayed here during SXSW and had a really ple...,4.5
1,"Great place, close enough to everything downto...",4.5
2,We had a great time in Austin staying at Paddy...,4.5
3,We had a great stay at Zen East for South By S...,4.5
4,I arrived late in the evening so did not meet ...,4.5


In [12]:
goodtrain_df = train_df[(train_df['review_scores_value'] >= 4) & (train_df['review_scores_value'] <= 5)]
oktrain_df = train_df[(train_df['review_scores_value'] >= 2.1) & (train_df['review_scores_value'] <= 3.9)]
badtrain_df = train_df[(train_df['review_scores_value'] >= 0) & (train_df['review_scores_value'] <= 2)]

goodtrain_df = goodtrain_df['comments']
oktrain_df = oktrain_df['comments']
badtrain_df = badtrain_df['comments']

goodtrain_df.head()

0    I stayed here during SXSW and had a really ple...
1    Great place, close enough to everything downto...
2    We had a great time in Austin staying at Paddy...
3    We had a great stay at Zen East for South By S...
4    I arrived late in the evening so did not meet ...
Name: comments, dtype: object

# Export

In [13]:
goodtrain_df.to_csv("goodcomments", index=False, compression='gzip')
oktrain_df.to_csv("okcomments", index=False, compression='gzip')
badtrain_df.to_csv("badcomments", index=False, compression='gzip')

/Users/nathanwong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
/Users/nathanwong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
/Users/nathanwong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package s